In [4]:
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [30]:
liste_variables1 = ['P19_FSCOL1824','P19_FSAL15P_TP', 'P19_FACTOCC15P', 'SNHMF20','P19_HSCOL1824', 'P19_HSAL15P_TP',
              'P19_HACTOCC15P', 'SNHMH20','CODGEO','EU_circo']

df = pd.read_csv('all_content_2020_v5.csv',encoding= "utf-8-sig",index_col='CODGEO',usecols=liste_variables1,
                 dtype = {'CODGEO' : 'str'})

In [32]:
df_index = df[df['EU_circo'] == 'Outre-Mer'].index
df = df.drop(df_index, axis=0)

In [33]:
df.tail()

,EU_circo,P19_HSCOL1824,P19_FSCOL1824,P19_HSAL15P_TP,P19_FSAL15P_TP,P19_HACTOCC15P,P19_FACTOCC15P,SNHMF20,SNHMH20
CODGEO,,,,,,,,,
95676,Île-de-France,9.000000,3.000000,9.000000,18.000000,118.000000,118.000000,NaN,NaN
95678,Île-de-France,13.833271,22.722538,10.009686,32.731779,187.262576,191.463491,NaN,NaN
95680,Île-de-France,604.672039,684.296128,556.539247,1053.744442,5395.267969,4589.996690,12.021008,12.284764
95682,Île-de-France,0.000000,2.000000,2.000000,8.000000,53.000000,46.000000,NaN,NaN
95690,Île-de-France,2.803725,5.744960,1.892451,13.691687,94.568961,81.612469,NaN,NaN


In [34]:
#Constitution des colonnes d'écarts, ajouter "*100" à la fin de chaque ligne pour avoir le format '10%' au lieu de '0.1'
df['Ecart H/F études supérieures'] = ((df['P19_HSCOL1824']- df['P19_FSCOL1824'])/df['P19_HSCOL1824'])
df['Ecart H/F temps partiels'] = ((df['P19_FSAL15P_TP']- df['P19_HSAL15P_TP'])/df['P19_FSAL15P_TP'])
df['Ecart H/F actifs occupés'] = ((df['P19_HACTOCC15P']- df['P19_FACTOCC15P'])/df['P19_HACTOCC15P'])
df['Ecart H/F Salaire net moyen'] = ((df['SNHMH20']- df['SNHMF20'])/df['SNHMH20'])

In [36]:
#Remplacement des infinite values par des Nan pour ne pas perdre de lignes(les infinite values gênent pour le machine learning)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [39]:
col_final = ['Ecart H/F études supérieures','Ecart H/F temps partiels','Ecart H/F actifs occupés', 
             'Ecart H/F Salaire net moyen']
variables_inegalites_HF = df[col_final]
variables_inegalites_HF.head()

,Ecart H/F études supérieures,Ecart H/F temps partiels,Ecart H/F actifs occupés,Ecart H/F Salaire net moyen
CODGEO,,,,
01001,0.015855,0.791777,0.147348,NaN
01002,0.518163,0.099163,0.130112,NaN
01004,-0.153232,0.655252,0.137532,0.194943
01005,-0.470588,0.835052,0.086957,NaN
01006,NaN,1.000000,0.214286,NaN


In [41]:
#Suppression des Nan 
variables_inegalites_HF.fillna(df.mean(), inplace=True)

In [43]:
variables_inegalites_HF.round(2)

,Ecart H/F études supérieures,Ecart H/F temps partiels,Ecart H/F actifs occupés,Ecart H/F Salaire net moyen
CODGEO,,,,
01001,0.02,0.79,0.15,0.16
01002,0.52,0.10,0.13,0.16
01004,-0.15,0.66,0.14,0.19
01005,-0.47,0.84,0.09,0.16
01006,-0.10,1.00,0.21,0.16
...,...,...,...,...
95676,0.67,0.50,0.00,0.16
95678,-0.64,0.69,-0.02,0.16
95680,-0.13,0.47,0.15,0.02


In [44]:
variables_inegalites_HF.isna().sum()

Ecart H/F études supérieures    0
Ecart H/F temps partiels        0
Ecart H/F actifs occupés        0
Ecart H/F Salaire net moyen     0
dtype: int64

In [45]:
variables_inegalites_HF.shape

(34826, 4)

In [46]:
import base64
from IPython.display import HTML

def create_download_link( df,filename,title):
    csv = df.to_csv(encoding="utf-8-sig")
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


file = "variables_inegalites_HF_v2.csv"
create_download_link(variables_inegalites_HF,file,"Download CSV file 1")